<h1> <center> Kaggle competition Submission</center> </h1>

## AMMI Bootcamp Kaggle competition

- This was a project assign students to develop a predictive model for wine prices with given features. 
- Predict the price of a bottle of wine based on a collection of over one hundred thousand reviews and other product features.


In [2]:
#### Import all necessary packages 
import numpy as np 
import pandas as pd 
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import style 
style.use("ggplot")
import warnings; warnings.filterwarnings("ignore")

In [4]:
### Import the data using pandas module read_csv
### Reading the training and testing datasets.
Train_data = pd.read_csv("train_competition.csv")
Test_data = pd.read_csv("test.csv")
Data.head(4)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,Portugal,This is a fine rich balanced wine. It has ripe...,Vila Santa Reserva,88.870874,20.0,Alentejano,NaN,NaN,NaN,NaN,NaN,PORTUGUESE RED,J. Portugal Ramos,32027
1,France,"A solid, chunky wine, with a structure that is...",NaN,88.041695,28.0,Bordeaux,Lalande de Pomerol,NaN,NaN,NaN,NaN,BORDEAUX-STYLE RED BLEND,Château Tour Grand Colombier,71079
2,France,"This is powerful and concentrated, with the hi...",NaN,94.085021,130.0,Bordeaux,Saint-Émilion,NaN,NaN,NaN,NaN,BORDEAUX-STYLE RED BLEND,Château Figeac,32440
3,US,"Rich, ripe and oaky, this Petite Sirah charms ...",Thompson Vineyard,89.869797,34.0,California,Santa Barbara County,Central Coast,NaN,NaN,Jaffurs 2010 Thompson Vineyard Petite Sirah (S...,PETITE SIRAH,Jaffurs,124405


In [6]:
## Check the number of missing values in the train dataset each feature. 
Train_data.isnull().sum()

country                      47
description                   0
designation               52266
points                        0
price                         0
province                     47
region_1                  28534
region_2                  99606
taster_name              109491
taster_twitter_handle    112810
title                     92811
variety                       1
winery                        0
id                            0
dtype: int64

In [154]:
### Check the number of missing values in the test set 
Test_data.isnull().sum()

index                        0
country                     17
description                  0
designation              24824
points                       0
price                    83210
province                    17
region_1                 13883
region_2                 47608
taster_name              52240
taster_twitter_handle    53841
title                    44424
variety                      0
winery                       0
id                           0
dtype: int64

In [7]:
### To check the feature characteristics of each column in the train dataset.
Train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175000 entries, 0 to 174999
Data columns (total 14 columns):
country                  174953 non-null object
description              175000 non-null object
designation              122734 non-null object
points                   175000 non-null float64
price                    175000 non-null float64
province                 174953 non-null object
region_1                 146466 non-null object
region_2                 75394 non-null object
taster_name              65509 non-null object
taster_twitter_handle    62190 non-null object
title                    82189 non-null object
variety                  174999 non-null object
winery                   175000 non-null object
id                       175000 non-null int64
dtypes: float64(2), int64(1), object(11)
memory usage: 18.7+ MB


In [8]:
### Dimensional checks of the dataframe 
def dimension_check(Data):
    ##print out the shape of the data.
    print("The shape of the dataset with row: ",Data.shape[0],"\n") 
    print("The shape of the dataset with columns: ",Data.shape[1],"\n")
    

In [9]:
#### The shape of the columns and rows in the dataset.
dimension_check(Data=Train_data)

The shape of the dataset with row:  175000 

The shape of the dataset with columns:  14 



In [10]:
#### The shape of the columns and rows in the dataset.
dimension_check(Data=Test_data)

The shape of the dataset with row:  83210 

The shape of the dataset with columns:  15 



## Preprocessing of data

- At this point of the analysis, we want to perform some preprocess techniques on the dataset. For instance: missing values. This is the cleaning stage of data modelling. We need to treat the missing values in each feature column of the dataframe for both training and test set. 

- We need to perform some missing values imputation or dropping of missing value. 

- Both training and test dataset must be treated differently. 

In [11]:
### Make a copy of the data 
Train_copy = Train_data.copy()
Test_copy = Test_data.copy()

In [12]:
### To treat the missing values for each dataset 
Train_copy.isnull().sum()

country                      47
description                   0
designation               52266
points                        0
price                         0
province                     47
region_1                  28534
region_2                  99606
taster_name              109491
taster_twitter_handle    112810
title                     92811
variety                       1
winery                        0
id                            0
dtype: int64

In [13]:
### Drop missing values with small number 
## Country, province, variety
Train_copy = Train_copy.dropna(subset=["country","province","variety"])
Train_copy.isnull().sum()

country                       0
description                   0
designation               52257
points                        0
price                         0
province                      0
region_1                  28486
region_2                  99558
taster_name              109486
taster_twitter_handle    112805
title                     92807
variety                       0
winery                        0
id                            0
dtype: int64

In [14]:
##### Drop features which are not important in the dataframe 
##### Define a function to drop  
def drop_features(Data,non_important):
    Data = Data.drop(non_important,axis="columns")
    return Data

In [15]:
Train_copy = drop_features(Data=Train_copy,non_important=['taster_name','taster_twitter_handle','id','title'])
Train_copy.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')

In [17]:
####Drop all non-informative features
Test_copy = drop_features(Data=Test_copy,non_important=['index','price','taster_name','taster_twitter_handle','id','title'])
Test_copy.columns

Index(['country', 'description', 'designation', 'points', 'province',
       'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')

In [19]:
### From the data information there was no information for those columns. So we had to replace with no location.
def missing_town(Data,feature):
    Data[feature] = Data[feature].fillna("No location")
    return Data 

In [20]:
Train_copy = missing_town(Data=Train_copy,feature="region_2")

In [21]:
Test_copy = missing_town(Data=Test_copy,feature="region_2")

In [23]:
### Check for the uniqueness in each feature. 
Features = ["designation","province","region_1","region_2","variety","winery"]
def Uniqueness(Data,Feature):
    ### 
    Unique_character = []
    Feature = []
    for i in Features:
        counts = Data[i].nunique()
        Unique_character.append(counts)
        Feature.append(i)
    Feat = pd.DataFrame({"Column_name": Feature,"Number_of_uniqueness":Unique_character})
    return Feat
#### Check for train set
Uniqueness(Data=Train_copy,Feature=Features)

,Column_name,Number_of_uniqueness
0,designation,37912
1,province,468
2,region_1,1278
3,region_2,19
4,variety,702
5,winery,16957


In [24]:
### Check for the uniqueness in each feature. 
## Check for test set
Uniqueness(Data=Test_copy,Feature=Features)

,Column_name,Number_of_uniqueness
0,designation,25487
1,province,423
2,region_1,1145
3,region_2,19
4,variety,622
5,winery,13962


In [27]:
Train_copy.isnull().sum()

country            0
description        0
designation    52257
points             0
price              0
province           0
region_1       28486
region_2           0
variety            0
winery             0
dtype: int64

In [25]:
Test_copy.isnull().sum()

country           17
description        0
designation    24824
points             0
province          17
region_1       13883
region_2           0
variety            0
winery             0
dtype: int64

In [29]:
### Filling the missing values with of the categorical features with mode 
def filling_missing(Data):
    for m in Data.columns:
        if Data[m].isnull().sum() > 0: 
            Data[m] = Data[m].fillna(Data[m].mode()[0])
    return Data 
        

In [30]:
Train_copy = filling_missing(Data=Train_copy)
Train_copy.isnull().sum()

country        0
description    0
designation    0
points         0
price          0
province       0
region_1       0
region_2       0
variety        0
winery         0
dtype: int64

In [31]:
Test_copy = filling_missing(Data=Test_copy)
Test_copy.isnull().sum()

country        0
description    0
designation    0
points         0
province       0
region_1       0
region_2       0
variety        0
winery         0
dtype: int64

### Feature Engineering 

- At this stage of the analysis, we want to perform some feature engineering before building the model. Both data have column feature with text.

In [32]:
### import the tensorflow framework and other text processing modules 
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers

In [33]:
y_label_train = Train_copy["price"]
X_labels_train = Train_copy.drop("price",axis="columns")

In [34]:
## Check for columns names of the input features
X_labels_train.columns

Index(['country', 'description', 'designation', 'points', 'province',
       'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')

In [35]:
## Check for columns names of the test set. 
Test_copy.columns

Index(['country', 'description', 'designation', 'points', 'province',
       'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')

In [36]:
### Check for the shape of the model. 
X_labels_train.shape, Test_copy.shape

((174952, 9), (83210, 9))

In [37]:
### Import the nltk package by using the word_tokenize module.
from nltk.tokenize import word_tokenize

,country,description,designation,points,province,region_1,region_2,variety,winery
0,Portugal,This is a fine rich balanced wine. It has ripe...,Vila Santa Reserva,88.870874,Alentejano,Napa Valley,No location,PORTUGUESE RED,J. Portugal Ramos
1,France,"A solid, chunky wine, with a structure that is...",Reserve,88.041695,Bordeaux,Lalande de Pomerol,No location,BORDEAUX-STYLE RED BLEND,Château Tour Grand Colombier
2,France,"This is powerful and concentrated, with the hi...",Reserve,94.085021,Bordeaux,Saint-Émilion,No location,BORDEAUX-STYLE RED BLEND,Château Figeac
3,US,"Rich, ripe and oaky, this Petite Sirah charms ...",Thompson Vineyard,89.869797,California,Santa Barbara County,Central Coast,PETITE SIRAH,Jaffurs
4,US,This wine is a unique in the state blend and f...,McKinley Springs Vineyard,89.017651,Washington,Horse Heaven Hills,Columbia Valley,ROSé,Syncline


In [63]:
X_labels_train[X_labels_train["designation"] == "..."].index

Int64Index([], dtype='int64')

In [66]:
Counter(Test_copy["region_2"])

Counter({'Napa': 5010,
         'No location': 47608,
         'Central Coast': 7788,
         'Sonoma': 6505,
         'Columbia Valley': 5478,
         'California Other': 1927,
         'Willamette Valley': 2098,
         'Finger Lakes': 1050,
         'Central Valley': 700,
         'Sierra Foothills': 961,
         'Long Island': 468,
         'Mendocino/Lake Counties': 729,
         'Napa-Sonoma': 898,
         'Washington Other': 384,
         'North Coast': 395,
         'New York Other': 99,
         'Oregon Other': 431,
         'Southern Oregon': 529,
         'South Coast': 152})

In [53]:
X_labels_train.isnull().sum()

country        0
description    0
designation    0
points         0
province       0
region_1       0
region_2       0
variety        0
winery         0
dtype: int64

In [54]:
Test_copy.isnull().sum()

country        0
description    0
designation    0
points         0
province       0
region_1       0
region_2       0
variety        0
winery         0
dtype: int64

In [69]:
def tokenize_process(Train,Test):
    ### In this part of the code, we perform some tokenization here. 
    points = Train[["points"]]
    points1 = Test[["points"]]
    Train = Train[['country', 'description', 'designation', 'province','region_1', 'region_2', 'variety', 'winery']]
    Test = Test[['country', 'description', 'designation', 'province','region_1', 'region_2', 'variety', 'winery']]
    vocabulary_size = 25000
    _word = lambda words: len(word_tokenize(words))
    Train_dict = {}
    Test_dict = {}
    for columns in Train.columns:
        Longest_sentence = max(Train[columns],key=_word)
        Longest_length_sentence = len(word_tokenize(Longest_sentence))
        Train_col = [one_hot(token,vocabulary_size) for token in Train[columns]]
        Test_col = [one_hot(token,vocabulary_size) for token in Test[columns]]
        Train_dict[columns] = pad_sequences(Train_col,Longest_length_sentence,padding="post")
        Test_dict[columns] = pad_sequences(Test_col,Longest_length_sentence,padding="post")
    ### Concatenate the features engineered.
    X_train = np.concatenate((Train_dict["country"],Train_dict["description"],Train_dict["designation"],Train_dict["province"],Train_dict["region_1"],Train_dict["region_2"],Train_dict["variety"],Train_dict["winery"],points.to_numpy()),axis=1)
    X_test = np.concatenate((Test_dict["country"],Test_dict["description"],Test_dict["designation"],Test_dict["province"],Test_dict["region_1"],Test_dict["region_2"],Test_dict["variety"],Test_dict["winery"],points1.to_numpy()),axis=1)
    return X_train, X_test

In [70]:
X_train, X_test = tokenize_process(Train=X_labels_train,Test=Test_copy)

In [71]:
### Check for dimension for both training and testing 
print(X_train.shape)
print(X_test.shape)

(174952, 213)
(83210, 213)


In [31]:
### Install the nltk packages
#import nltk
#nltk.download("punkt")

In [72]:
### Import the packages in tensorflow framework.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers

In [73]:
### Store the correct model.
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [74]:
## Define our networks using the Sequential in models modules
embed_dimension = 25
Model = Sequential([
    layers.Embedding(input_dim = 25000,output_dim=embed_dimension,input_length=X_train.shape[1]),
    layers.Flatten(),
    layers.Dense(13,activation="relu"),
    layers.Dense(18,activation="relu"),
    layers.Dense(1,activation="linear")
])

In [75]:
### Set our optimizer using Adam with their given parameters 
optimizer = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
Model.compile(optimizer=optimizer,loss="mean_squared_error",metrics=["accuracy"])

In [76]:
Model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 213, 25)           625000    
_________________________________________________________________
flatten (Flatten)            (None, 5325)              0         
_________________________________________________________________
dense (Dense)                (None, 13)                69238     
_________________________________________________________________
dense_1 (Dense)              (None, 18)                252       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 19        
Total params: 694,509
Trainable params: 694,509
Non-trainable params: 0
_________________________________________________________________


In [78]:
###Fit the train data to our model by training it for 50 epoch with batch size of 20
Model.fit(X_train, y_label_train, epochs=20, batch_size = 10, validation_split=0.3, callbacks=callbacks_list, shuffle=True)

Train on 122466 samples, validate on 52486 samples
Epoch 1/20
122430/122466 [============================>.] - ETA: 0s - loss: 44.3336 - accuracy: 0.0000e+00
Epoch 00001: val_accuracy did not improve from 0.00000
122466/122466 [==============================] - 69s 566us/sample - loss: 44.3263 - accuracy: 0.0000e+00 - val_loss: 541.7136 - val_accuracy: 0.0000e+00
Epoch 2/20
122390/122466 [============================>.] - ETA: 0s - loss: 47.2112 - accuracy: 0.0000e+00
Epoch 00002: val_accuracy did not improve from 0.00000
122466/122466 [==============================] - 71s 581us/sample - loss: 47.2524 - accuracy: 0.0000e+00 - val_loss: 551.7092 - val_accuracy: 0.0000e+00
Epoch 3/20
122410/122466 [============================>.] - ETA: 0s - loss: 42.3497 - accuracy: 0.0000e+00
Epoch 00003: val_accuracy did not improve from 0.00000
122466/122466 [==============================] - 72s 590us/sample - loss: 42.3379 - accuracy: 0.0000e+00 - val_loss: 534.9016 - val_accuracy: 0.0000e+00
Epoc

In [79]:
pred = Model.predict(X_test)

In [81]:
Submission1 = Test_data[["id"]]

In [82]:
Submission1["price"] = pred

In [83]:
Submission.head()

NameError: name 'Submission' is not defined

In [197]:
Submission1.head()

,id,price
0,0,100.261703
1,1,39.950363
2,2,60.686573
3,3,23.301657
4,4,13.390927


In [198]:
Submission1.to_csv("Submission.csv",index=False)